# Model training with Automated ML

In the cell below, import all the dependencies that you will need to complete the project.

In [ ]:
!pip install 'azureml-sdk'
!pip install 'azureml-sdk[notebooks]'

In [1]:
# Imports
import pandas as pd

In [1]:
# Azure ML Imports
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core import Workspace, Experiment
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.widgets import RunDetails
from azureml.train.automl import AutoMLConfig

## Workspace

In [ ]:
ws = Workspace.from_config()

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

## Compute

Create a remote GPU compute cluster for model training

In [ ]:
# Choose a name for your CPU cluster
gpu_cluster_name = "gpu-cluster"

# Verify that cluster does not exist already
try:
    gpu_cluster = ComputeTarget(workspace=ws, name=gpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_NC24',
                                                           max_nodes=10)
    gpu_cluster = ComputeTarget.create(ws, gpu_cluster_name, compute_config)

gpu_cluster.wait_for_completion(show_output=True)

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [ ]:


# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://www.kaggle.com/datamunge/sign-language-mnist"

found = False
key = "sign-language-mnist"
description_text = "sign Language MNIST"

if key in ws.datasets.keys(): 
    found = True
    ds = ws.datasets[key] 

if not found:
    from azureml.data.dataset_factory import TabularDatasetFactory
    datastore_path = "https://github.com/emanbuc/ASL-Recognition-Deep-Learning/raw/main/datasets/sign-language-mnist/sign_mnist_train/sign_mnist_train.csv"
    ds = TabularDatasetFactory.from_delimited_files(path=datastore_path,header=True)       
    #Register Dataset in Workspace
    ds = ds.register(workspace=ws,
                                   name=key,
                                   description=description_text)


df = ds.to_pandas_dataframe()
df.describe()

## AutoML Configuration

**max_concurrent_iterations** : 10

AmlCompute clusters support one interation running per node. For multiple AutoML experiment parent runs executed in parallel on a single AmlCompute cluster, the sum of the max_concurrent_iterations values for all experiments should be less than or equal to the maximum number of nodes. Otherwise, runs will be queued until nodes are available.
Set to 10 as number of node in compute cluster.

**iteration_timeout_minutes** : 10

Maximum time in minutes that each iteration can run for before it terminates. 30 minutes to avoid Lab timeout.

**experiment_timeout_hours**: 0.5

Experiment must end before lab timeout

**enable_early_stopping**: true

Whether to enable early termination if the score is not improving in the short term. Set to True to avoid waste time. We don't need to try every possible iteration in this demo experiment.

**enable_onnx_compatible_models**: True

Whether to enable or disable enforcing the ONNX-compatible models. Must be True to anable deploy on ONNX runtime.




In [ ]:
automl_settings = {
    "experiment_timeout_hours" : 0.5,
    "enable_early_stopping" : True,
    "iteration_timeout_minutes": 10,
    "max_concurrent_iterations": 10,
    "enable_onnx_compatible_models": True
}

automl_config = AutoMLConfig(
    debug_log='automl_errors.log',
    compute_target=gpu_cluster,
    task='classification',
    primary_metric='accuracy',
    training_data= ds,
    label_column_name='label',
    **automl_settings)

In [ ]:
# Submit AutoML Experiment
experiment_name = 'ASL-DeepLearning-AutoML'
exp_automl = Experiment(workspace=ws, experiment_name)
automl_run = experiment.submit(automl_config)

## Run Details

In [ ]:
automl_run = automl_experiment.submit(automl_config)
RunDetails(automl_run).show()
automl_run.wait_for_completion(show_output=True)

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [ ]:
best_automl_run, auto_ml_fitted_model = automl_run.get_output()
print(best_automl_run)
print(auto_ml_fitted_model)

In [ ]:
vc=auto_ml_fitted_model.steps[1][1]
vc.named_estimators

In [ ]:
#TODO: Save the best model
model = best_automl_run.register_model(model_name='automl-model', model_path='outputs/model.pkl')

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service